# Corona Map Reduce

In [1]:
import numpy as np

import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

In [2]:
#pandas is a powerful Python data analysis toolkit
#fast, flexible, and expressive data structures designed to make 
#working with relational or labeled data
df = pd.read_json("coronaextrausa0.json")
df.head()
#After testing the source many are posted by advertising agencies

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,NaN,2016-11-25 22:05:05,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,16,False,NaN,802271920146128896,802271920146128896,...,NaN,NaN,NaN,12,False,NaN,"<a href=""http://percolate.com"" rel=""nofollow"">...",Que comience la temporada de compartir momento...,False,"{'follow_request_sent': False, 'has_extended_p..."
1,NaN,NaN,2016-11-25 22:00:08,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,38,False,NaN,802270673183731712,802270673183731712,...,NaN,NaN,NaN,22,False,NaN,"<a href=""http://percolate.com"" rel=""nofollow"">...",Find your tree yet? https://t.co/C7VCHj2e9b,False,"{'follow_request_sent': False, 'has_extended_p..."
2,NaN,NaN,2016-11-20 16:06:14,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,13,False,NaN,800369672776466432,800369672776466432,...,NaN,NaN,NaN,5,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Cuando te dejas llevar por la música ocurren c...,False,"{'follow_request_sent': False, 'has_extended_p..."
3,NaN,NaN,2016-11-20 16:05:43,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,29,False,NaN,800369540844679168,800369540844679168,...,NaN,NaN,NaN,8,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",That moment when your song takes you to that h...,False,"{'follow_request_sent': False, 'has_extended_p..."
4,NaN,NaN,2016-11-11 21:22:18,"{'symbols': [], 'user_mentions': [], 'hashtags...",{'media': [{'expanded_url': 'https://twitter.c...,15,False,NaN,797187721127149568,797187721127149568,...,NaN,NaN,NaN,9,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#FightNight done right. Catch Kovalev vs Ward ...,False,"{'follow_request_sent': False, 'has_extended_p..."


In [3]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'extended_entities', 'favorite_count', 'favorited', 'geo', 'id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'text', 'truncated', 'user'],
      dtype='object')

In [4]:
df.dtypes

contributors                        float64
coordinates                         float64
created_at                   datetime64[ns]
entities                             object
extended_entities                    object
favorite_count                        int64
favorited                              bool
geo                                 float64
id                                    int64
id_str                                int64
in_reply_to_screen_name              object
in_reply_to_status_id               float64
in_reply_to_status_id_str           float64
in_reply_to_user_id                 float64
in_reply_to_user_id_str             float64
is_quote_status                        bool
lang                                 object
place                               float64
possibly_sensitive                  float64
quoted_status                        object
quoted_status_id                    float64
quoted_status_id_str                float64
retweet_count                   

In [5]:
ndf = df[['text', 'created_at', 'retweet_count']]

ndf.head(5)

,text,created_at,retweet_count
0,Que comience la temporada de compartir momento...,2016-11-25 22:05:05,12
1,Find your tree yet? https://t.co/C7VCHj2e9b,2016-11-25 22:00:08,22
2,Cuando te dejas llevar por la música ocurren c...,2016-11-20 16:06:14,5
3,That moment when your song takes you to that h...,2016-11-20 16:05:43,8
4,#FightNight done right. Catch Kovalev vs Ward ...,2016-11-11 21:22:18,9


In [6]:
df2 = pd.read_json("coronaextrausa1.json")
ndf2 = df2[['text', 'created_at', 'retweet_count']]
ndf2.head(5)

,text,created_at,retweet_count
0,@ktuck_91 Safe travels on your journey. Don't ...,2016-07-08 21:56:48,0
1,@quetallitabusca Cómo debe de ser. Salud!,2016-07-07 15:35:00,0
2,@adriana02022879 En el mar la vida es más sabr...,2016-07-07 15:30:08,0
3,@JoshTaerk And #BastilleDay coming up next wee...,2016-07-05 20:59:12,1
4,@the1andonlypaco Permission granted.,2016-07-05 20:58:51,1


In [7]:
df3 = pd.read_json("coronaextrausa2.json")
ndf3 = df3[['text', 'created_at', 'retweet_count']]
ndf3.head(5)

,text,created_at,retweet_count
0,@ToddyFur @chalkybear13 Salud!,2016-04-18 20:19:38,0
1,@chamberzone_ Here's to you. Salud!,2016-04-18 20:18:16,0
2,"@jaredcooper4 More beach vibes, please!",2016-04-18 20:18:01,0
3,@solita_valencia Gotta file some limes. #TaxDay,2016-04-18 20:17:36,0
4,@KodiboyLaurie That summer feeling.,2016-04-18 20:17:16,1


In [8]:
df4 = pd.read_json("coronaextrausa3.json")
ndf4 = df4[['text', 'created_at', 'retweet_count']]
ndf4.head(5)

,text,created_at,retweet_count
0,¡Pero que vista!\nMuéstranos tu #ViewFromMyCo...,2016-01-27 17:36:42,7
1,Change the forecast of your day with #CoronaEx...,2016-01-25 04:10:50,10
2,Cualquier día del año es perfecto para disfrut...,2016-01-25 01:58:26,9
3,Dooooown - Set - Hut - Hut - #CoronaExtra. htt...,2016-01-22 19:00:03,13
4,¡Ya estamos listos para el Gran Juego! #Corona...,2016-01-22 17:00:05,18


In [9]:
df5 = pd.read_json("coronaextrausa4.json")
ndf5 = df5[['text', 'created_at', 'retweet_count']]
ndf5.head(5)

,text,created_at,retweet_count
0,CoronaRita &gt; margarita. \nFave this to shak...,2015-07-23 21:04:52,11
1,Un amor de verano que no quieres que termine. ...,2015-07-21 23:04:34,11
2,This is one summer fling we never want to end....,2015-07-21 21:08:30,26
3,Receta para el verano perfecto:\nRelájate.\nRe...,2015-07-18 07:23:45,13
4,Summer vacation recipe: \nRelax. \nRecenter. \...,2015-07-17 21:00:05,13


In [10]:
df6 = pd.read_json("coronaextrausa5.json")
ndf6 = df6[['text', 'created_at', 'retweet_count']]
ndf6.head(5)

,text,created_at,retweet_count
0,@Buzzfeed’s 11 reasons why summer is better f...,2014-05-28 22:30:05,13
1,Show us how you enjoy your #CoronaSummer. / E...,2014-05-27 20:00:11,20
2,There’s always room for something Extra at the...,2014-05-26 20:00:09,27
3,Long weekend essentials. The only thing missin...,2014-05-23 22:30:06,35
4,How are you keeping cool this #CoronaSummer? ¿...,2014-05-22 20:00:13,7


In [11]:
df7 = pd.read_json("coronaextrausa6.json")
ndf7 = df7[['text', 'created_at', 'retweet_count']]
ndf7.head(5)

,text,created_at,retweet_count
0,Retweet if you think this should be a universa...,2013-05-07 22:38:49,45
1,Share your Corona Summer pictures &amp; one mi...,2013-05-06 20:58:54,3
2,"Happy Corona de Mayo! Friends, food, fun and C...",2013-05-06 03:32:29,19
3,Get your Cinco de Mayo weekend started with fu...,2013-05-03 22:36:29,26
4,Cinco de Mayo is almost here! Here’s something...,2013-05-03 18:31:37,10


In [55]:
cdf = pd.concat([ndf,ndf2], ignore_index=True)
cdf2 = pd.concat([cdf,ndf3], ignore_index=True)
cdf3 = pd.concat([cdf2,ndf4], ignore_index=True)
cdf4 = pd.concat([cdf3,ndf5], ignore_index=True)
cdf5 = pd.concat([cdf4,ndf6], ignore_index=True)
corona_map_reduce = pd.concat([cdf5,ndf7], ignore_index=True)
corona_map_reduce_text = corona_map_reduce[["text"]]
corona_map_reduce_text
filename = 'corona3.csv'
corona_map_reduce_text.to_csv(filename, index=False, header=False, encoding='utf-8')

# 20th Fox Map Reduce

In [13]:
#Read in 16 datasets of json from fox
ff1 = pd.read_json("20thcenturyfox0.json")
ff2 = pd.read_json("20thcenturyfox1.json")
ff3 = pd.read_json("20thcenturyfox2.json")
ff4 = pd.read_json("20thcenturyfox3.json")
ff5 = pd.read_json("20thcenturyfox4.json")
ff6 = pd.read_json("20thcenturyfox5.json")
ff7 = pd.read_json("20thcenturyfox6.json")
ff8 = pd.read_json("20thcenturyfox7.json")
ff9 = pd.read_json("20thcenturyfox8.json")
ff10 = pd.read_json("20thcenturyfox9.json")
ff11 = pd.read_json("20thcenturyfox10.json")
ff12 = pd.read_json("20thcenturyfox11.json")
ff13 = pd.read_json("20thcenturyfox12.json")
ff14 = pd.read_json("20thcenturyfox13.json")
ff15 = pd.read_json("20thcenturyfox14.json")
ff16 = pd.read_json("20thcenturyfox15.json")

In [14]:
#Map Reduce on tweets
nff1 = ff1[['text', 'created_at', 'retweet_count']]
nff2 = ff2[['text', 'created_at', 'retweet_count']]
nff3 = ff3[['text', 'created_at', 'retweet_count']]
nff4 = ff4[['text', 'created_at', 'retweet_count']]
nff5 = ff5[['text', 'created_at', 'retweet_count']]
nff6 = ff6[['text', 'created_at', 'retweet_count']]
nff7 = ff7[['text', 'created_at', 'retweet_count']]
nff8 = ff8[['text', 'created_at', 'retweet_count']]
nff9 = ff9[['text', 'created_at', 'retweet_count']]
nff10 = ff10[['text', 'created_at', 'retweet_count']]
nff11 = ff11[['text', 'created_at', 'retweet_count']]
nff12 = ff12[['text', 'created_at', 'retweet_count']]
nff13 = ff13[['text', 'created_at', 'retweet_count']]
nff14 = ff14[['text', 'created_at', 'retweet_count']]
nff15 = ff15[['text', 'created_at', 'retweet_count']]
nff16 = ff16[['text', 'created_at', 'retweet_count']]

In [56]:
#Dataset Concatonation
fox_map_reduced = pd.concat([nff1,nff2,nff3,nff4,nff5,nff6,nff7,nff8,nff9,nff10,nff11,nff12,nff13,nff14,nff15,nff16], ignore_index=True)
fox_map_reduce = pd.concat([nff1,nff2,nff3], ignore_index=True)

fox_map_reduced_text = fox_map_reduced[["text"]]
#fox_map_reduced_text


In [20]:
#fox_count = fox_map_reduce['text'].str.lower().str.split()
#fox_count.dtypes
#fox_count
#attempted tokenization

In [21]:
#fmr = fox_map_reduce.text.apply(lambda x: pd.value_counts(x.split(" "))).sum()
#fmr
#attempted tokenization

In [45]:
file = 'fox.csv'
fox_map_reduced_text.to_csv(file, index=False, header=False, encoding='utf-8')

In [58]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
 
text_pos = []
labels_pos = []
filename = "fox.csv"
with open(filename, encoding="utf8") as f:
    for i in f: 
        text_pos.append(i) 
        labels_pos.append('pos')
        
text_neg = []
labels_neg = []
filename2 = "corona3.csv"
with open(filename2, encoding="utf8") as f:
    for i in f: 
        text_neg.append(i)
        labels_neg.append('neg')
        


In [59]:
training_text = text_pos[:int((.8)*len(text_pos))] + text_neg[:int((.8)*len(text_neg))]
training_labels = labels_pos[:int((.8)*len(labels_pos))] + labels_neg[:int((.8)*len(labels_neg))]

test_text = text_pos[int((.8)*len(text_pos)):] + text_neg[int((.8)*len(text_neg)):]
test_labels = labels_pos[int((.8)*len(labels_pos)):] + labels_neg[int((.8)*len(labels_neg)):]


#80% training data--20% testing data 


In [60]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    max_features = 85
)

In [61]:
#Next, we use the fit_transform() method to transform our corpus data into feature vectors. Since the input needed is a list of strings, we concatenate all of our training and test data.
features = vectorizer.fit_transform(
    training_text + test_text)

features_nd = features.toarray() # for easy use

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test   = train_test_split(
        features_nd[0:len(training_text)], 
        training_labels,
        train_size=0.80, 
        random_state=1234)

# Linear Classifier

In [63]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [64]:
log_model = log_model.fit(X=X_train, y=y_train)

In [65]:
y_pred = log_model.predict(X_test)

In [71]:
import nltk

def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

print(format_sentence("The cat is very cute"))

{'The': True, 'cat': True, 'is': True, 'very': True, 'cute': True}


In [80]:
insults = pd.read_csv("insults.csv")
#insults.head()
imr = insults[["Comment"]]
#imr.head()
file = 'imr.csv'
fox_map_reduced_text.to_csv(file, index=False, header=False, encoding='utf-8')

In [81]:
pos = []
with open("fox.csv", encoding="utf8") as f:
    for i in f: 
        pos.append([format_sentence(i), 'pos'])
        
neg = []
with open("imr.csv",encoding="utf8") as f:
    for i in f: 
        neg.append([format_sentence(i), 'neg'])


In [82]:
training = pos[:int((.8)*len(pos))] + neg[:int((.8)*len(neg))]
test = pos[int((.8)*len(pos)):] + neg[int((.8)*len(neg)):]

In [83]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(training)

In [84]:
classifier.show_most_informative_features()

Most Informative Features
       //t.co/qKk2WigMXZ = None              neg : pos    =      1.0 : 1.0
                   their = True              neg : pos    =      1.0 : 1.0
                    Risk = True              neg : pos    =      1.0 : 1.0
                   Guild = None              neg : pos    =      1.0 : 1.0
            endorsements = True              neg : pos    =      1.0 : 1.0
                      2. = True              neg : pos    =      1.0 : 1.0
       //t.co/bLuTPFKIkt = None              neg : pos    =      1.0 : 1.0
               slashfilm = None              neg : pos    =      1.0 : 1.0
                   Proud = None              neg : pos    =      1.0 : 1.0
                    sᴉɥʇ = None              neg : pos    =      1.0 : 1.0


In [91]:
example1 = "Don't get lost in "

print(classifier.classify(format_sentence(example1)))

pos
